In [2]:
# Construction of dataset

import os
from xml.dom import minidom


alignment_folder = "reference-alignment/"

# Load reference alignments 
def load_alignments(folder):
    alignments = []
    for f in os.listdir(folder):
        doc = minidom.parse(folder + f)
        ls = list(zip(doc.getElementsByTagName('entity1'), doc.getElementsByTagName('entity2')))
        alignments.extend([(a.getAttribute('rdf:resource'), b.getAttribute('rdf:resource')) for (a,b) in ls])
    return alignments
        
reference_alignments = load_alignments(alignment_folder)

In [87]:

class Ontology():
    def __init__(self, ontology):
        self.ontology = ontology
        self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement
        self.object_prop = "http://www.w3.org/2002/07/owl#ObjectProperty"
        self.data_prop = "http://www.w3.org/2002/07/owl#DatatypeProperty"
    
    def get_child_node(self, element, tag):
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]
        
    def has_attribute_value(self, element, attribute, value):
        return True if element.getAttribute(attribute) == value else False
    
    def get_triples(self):
        self.ontology_obj
    
    def filter_null(self, data):
        return [el for el in data if el]
    
    def extract_ID(self, element):
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        return element_id.split("#")[-1]
    
    def parse_classes(self):
        return self.get_child_node(self.root, 'owl:Class')
    
    def classes(self):
        classes = [self.extract_ID(el) for el in self.parse_classes()]
        return self.filter_null(classes)

    def parse_object_properties(self):
        obj_properties = [el for el in self.get_child_node(self.root, 'owl:ObjectProperty')]
        fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_obj_properties = [el for el in fn_obj_properties if type(el)==minidom.Element and 
            self.has_attribute_value(self.get_child_node(el, "rdf:type")[0], "rdf:resource", self.object_prop)]
        inv_fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_obj_properties = [el for el in inv_fn_obj_properties if type(el)==minidom.Element and 
            self.has_attribute_value(self.get_child_node(el, "rdf:type")[0], "rdf:resource", self.object_prop)]
        return obj_properties + fn_obj_properties + inv_fn_obj_properties
    
    def object_properties(self):
        obj_props = [self.extract_ID(el) for el in self.parse_object_properties()]
        return list(set(self.filter_null(obj_props)))

    def load_ontology(self):    
        onto = get_ontology(self.ontology).load()
        return onto

ontology = "conference_ontologies/conference.owl"

ont = Ontology(ontology)
